In [1]:
import sys
from typing import Literal

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-09 18:40:12.923007: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-09 18:40:12.929393: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-09 18:40:13.055069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 18:40:13.055200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 18:40:13.100469: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
n_samples = 32
sampler = EquationSampler(BASE_TOKENS, 16, 2)
(
    sequences,
    entropies,
    log_probs,
    counters,
    lengths,
    sequence_mask,
) = sampler.sample_sequence(n_samples, repeat=1)

array([[4, 7, 3, 7, 0, 9, 5, 3, 9, 8, 9, 8],
       [0, 8, 5, 2, 2, 0, 2, 8, 8, 8, 9, 8],
       [5, 0, 8, 9, 4, 5, 7, 4, 0, 4, 8, 8],
       [4, 2, 0, 2, 9, 3, 7, 8, 8, 8, 8, 8],
       [7, 9, 1, 8, 3, 7, 5, 5, 2, 8, 9, 8],
       [7, 7, 0, 2, 8, 0, 1, 8, 8, 9, 8, 9],
       [5, 0, 4, 1, 8, 1, 8, 7, 8, 3, 8, 9],
       [0, 4, 4, 4, 1, 0, 9, 8, 0, 9, 8, 9],
       [2, 1, 4, 4, 4, 0, 9, 3, 8, 8, 9, 8],
       [2, 7, 7, 5, 5, 8, 7, 0, 7, 9, 9, 9],
       [5, 7, 9, 2, 2, 3, 0, 9, 8, 8, 8, 8],
       [2, 7, 6, 4, 4, 4, 8, 1, 6, 8, 9, 8],
       [0, 9, 0, 0, 1, 1, 9, 9, 8, 8, 8, 9],
       [1, 2, 6, 7, 1, 8, 0, 8, 8, 9, 9, 8],
       [7, 9, 6, 9, 7, 0, 7, 9, 7, 9, 4, 3],
       [5, 2, 1, 4, 7, 7, 1, 9, 9, 9, 9, 8],
       [5, 0, 0, 5, 2, 1, 8, 8, 8, 8, 9, 8],
       [0, 1, 2, 4, 3, 1, 8, 8, 9, 9, 8, 8],
       [0, 5, 2, 3, 3, 4, 8, 9, 9, 9, 8, 9],
       [4, 7, 2, 5, 2, 4, 4, 8, 0, 9, 9, 8],
       [0, 8, 9, 3, 2, 9, 7, 9, 6, 8, 2, 9],
       [5, 5, 2, 5, 5, 7, 5, 1, 8, 8, 9, 9],
       [0,

In [3]:
sequences = TensorExpress(sequences)

In [4]:
sequences.astype(tf.int32)

<tf.Tensor: shape=(32, 12), dtype=int32, numpy=
array([[4, 7, 3, 7, 0, 9, 5, 3, 9, 8, 9, 8],
       [0, 8, 5, 2, 2, 0, 2, 8, 8, 8, 9, 8],
       [5, 0, 8, 9, 4, 5, 7, 4, 0, 4, 8, 8],
       [4, 2, 0, 2, 9, 3, 7, 8, 8, 8, 8, 8],
       [7, 9, 1, 8, 3, 7, 5, 5, 2, 8, 9, 8],
       [7, 7, 0, 2, 8, 0, 1, 8, 8, 9, 8, 9],
       [5, 0, 4, 1, 8, 1, 8, 7, 8, 3, 8, 9],
       [0, 4, 4, 4, 1, 0, 9, 8, 0, 9, 8, 9],
       [2, 1, 4, 4, 4, 0, 9, 3, 8, 8, 9, 8],
       [2, 7, 7, 5, 5, 8, 7, 0, 7, 9, 9, 9],
       [5, 7, 9, 2, 2, 3, 0, 9, 8, 8, 8, 8],
       [2, 7, 6, 4, 4, 4, 8, 1, 6, 8, 9, 8],
       [0, 9, 0, 0, 1, 1, 9, 9, 8, 8, 8, 9],
       [1, 2, 6, 7, 1, 8, 0, 8, 8, 9, 9, 8],
       [7, 9, 6, 9, 7, 0, 7, 9, 7, 9, 4, 3],
       [5, 2, 1, 4, 7, 7, 1, 9, 9, 9, 9, 8],
       [5, 0, 0, 5, 2, 1, 8, 8, 8, 8, 9, 8],
       [0, 1, 2, 4, 3, 1, 8, 8, 9, 9, 8, 8],
       [0, 5, 2, 3, 3, 4, 8, 9, 9, 9, 8, 9],
       [4, 7, 2, 5, 2, 4, 4, 8, 0, 9, 9, 8],
       [0, 8, 9, 3, 2, 9, 7, 9, 6, 8, 2, 9],
       

In [ ]:
tf.cast(sequences, dtype=tf.float32)

In [ ]:
tf.reduce_any(tf.reduce_all(sequence_mask, axis=1))